In [17]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu 

In [18]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-10 22:18:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-07-10 22:18:13 (1.18 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [20]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

### Create DataFrames to match tables

In [21]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()

+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306|yoomall 5M Antenn...| 2015-08-31|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678|Hosa GPM-103 3.5m...| 2015-08-31|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099|Channel Master Ti...| 2015-08-31|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257|LIMTECH Wall char...| 2015-08-31|          1|       

In [22]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = review_df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             6|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             5|
|    5453476|             2|
|   12406466|             3|
|   48146680|             1|
|    5953456|             1|
|   16457323|             5|
|   24540309|             1|
|   43366878|             2|
|   28796717|             1|
|   28258386|             1|
|   26079415|             2|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = review_df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|7793224531|Monster Equivalen...|
|9985975413|Mobile Excess Dis...|
|B000000O3J|Micro Cassette 60...|
|B00000JYIZ|Aiwa NSX-MT720 Ho...|
|B00001W0DC|Sony MDR-V250V Mo...|
|B00003CWDC|Monster Cable MMS...|
|B00003CWGY|Monster Z200i-2M ...|
|B00004WLM5|Aiwa FRA150 AM/FM...|
|B00004Y2XB|Monster Cable I10...|
|B00004Y2Z1|Monster Cable MV2...|
|B00004Y3TH|Monster Cable Ent...|
|B00004Z5AM|Kodak 1038769 CD-...|
|B00005LABY|Sharp CD-CH1500 C...|
|B00005NIMS|Memorex Dr. Clean...|
|B00005NSJS|Aiwa XP-SP920UAB3...|
|B00006356Z|MTX Audio MP41B 4...|
|B00006453P|Leslie Dame CD-22...|
|B000068V32|PDA Plus Leather ...|
|B00006ANWY|KLH M365T 6.5" 3-...|
|B00006ANYF|Philips FWC788 3-...|
+----------+--------------------+
only showing top 20 rows



In [23]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982| 2015-08-31|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953| 2015-08-31|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933| 2015-08-31|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886| 2015-08-31|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381| 2015-08-31|
|R1DGA6UQIVLKZ7|    2781942|B007B5V092|     152891509| 2015-08-31|
| RLQT3V8SMNIBH|     707292|B00IODHGVG|     717335277| 2015-08

In [24]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = review_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [38]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cfps9m1rgnqb.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Hanaacleveland101", 
          "driver":"org.postgresql.Driver"}
        

In [40]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)